In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/beijing/Beijing.csv


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_percentage_error
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
import pickle
warnings.filterwarnings('ignore')

# Data preprocessing function remains the same
def preprocess_data(df):
    """Preprocess the air quality data for SARIMAX modeling"""
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    df.set_index('datetime', inplace=True)
    df.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
    df = df.fillna(method='ffill')
    
    wind_dict = {
        'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5, 
        'E': 90, 'ESE': 112.5, 'SE': 135, 'SSE': 157.5,
        'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
        'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5
    }
    df['wd'] = df['wd'].map(wind_dict)
    
    df['hour_sin'] = np.sin(df.index.hour * (2 * np.pi / 24))
    df['hour_cos'] = np.cos(df.index.hour * (2 * np.pi / 24))
    
    return df

# Feature preparation function remains the same
def prepare_features(df):
    """Prepare features for SARIMAX modeling"""
    exog_features = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 
                     'DEWP', 'RAIN', 'wd', 'WSPM', 'hour_sin', 'hour_cos']
    
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(
        scaler.fit_transform(df[exog_features]),
        columns=exog_features,
        index=df.index
    )
    
    pm25_scaled = scaler.fit_transform(df[['PM2.5']])
    df_scaled['PM2.5'] = pm25_scaled
    
    return df_scaled, scaler, exog_features

# Enhanced evaluation metrics
def calculate_metrics(y_true, y_pred):
    """Calculate comprehensive performance metrics"""
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    r2 = r2_score(y_true, y_pred)
    
    metrics = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE (%)': mape,
        'R² Score': r2
    }
    
    return metrics

def create_visualizations(test_data, predictions, metrics):
    """Create comprehensive visualizations for model evaluation"""
    # Create figure with secondary y-axis
    fig = make_subplots(rows=3, cols=1,
                       subplot_titles=('PM2.5 Prediction: Actual vs Predicted',
                                     'Prediction Error Over Time',
                                     'Error Distribution'),
                       vertical_spacing=0.12,
                       row_heights=[0.4, 0.3, 0.3])

    # Actual vs Predicted
    fig.add_trace(
        go.Scatter(x=test_data.index, y=test_data['PM2.5'],
                  name="Actual", line=dict(color="blue")),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=test_data.index, y=predictions,
                  name="Predicted", line=dict(color="red")),
        row=1, col=1
    )

    # Error over time
    errors = test_data['PM2.5'] - predictions
    fig.add_trace(
        go.Scatter(x=test_data.index, y=errors,
                  name="Prediction Error",
                  line=dict(color="green")),
        row=2, col=1
    )

    # Error distribution
    fig.add_trace(
        go.Histogram(x=errors, name="Error Distribution",
                    nbinsx=50),
        row=3, col=1
    )

    # Update layout
    fig.update_layout(height=1000, width=1000,
                     showlegend=True,
                     title_text="SARIMAX Model Performance Analysis")
    
    return fig

# Enhanced SARIMAX training function
def train_sarimax(endog, exog, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24)):
    """Train SARIMAX model with specified parameters"""
    model = SARIMAX(
        endog,
        exog=exog,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    
    results = model.fit(disp=False, method='lbfgs', maxiter=50)
    return results

# Function to save the model
def save_model(model, filename='sarimax_model.pkl'):
    """Save the SARIMAX model to a file"""
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved to {filename}")

# Function to load the model
def load_model(filename='sarimax_model.pkl'):
    """Load a SARIMAX model from a file"""
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    print(f"Model loaded from {filename}")
    return model

# Main execution function
def run_sarimax_analysis(data_path):
    """Run complete SARIMAX analysis pipeline"""
    # Load and prepare data
    df = pd.read_csv(data_path)
    df = df[:70000]
    df_processed = preprocess_data(df)
    df_scaled, scaler, exog_features = prepare_features(df_processed)
    
    # Split data
    train_size = int(len(df_scaled) * 0.8)
    train_data = df_scaled[:train_size]
    test_data = df_scaled[train_size:]
    
    # Train model
    print("Training SARIMAX model...")
    model_results = train_sarimax(
        train_data['PM2.5'],
        train_data[exog_features],
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 24)
    )
    
    # Save the trained model
    save_model(model_results)
    
    # Make predictions
    print("Making predictions...")
    predictions = model_results.predict(
        start=len(train_data),
        end=len(df_scaled)-1,
        exog=test_data[exog_features]
    )
    
    # Calculate metrics
    metrics = calculate_metrics(test_data['PM2.5'], predictions)
    
    fig = create_visualizations(test_data, predictions, metrics)
    
    return model_results, metrics, fig, scaler

# Code to make predictions on new data
def make_prediction_on_new_data(new_data_path, scaler, model_filename='sarimax_model.pkl'):
    """Load model and make predictions on new data"""
    # Load the saved model
    model = load_model(model_filename)
    
    # Load and preprocess new data
    new_data = pd.read_csv(new_data_path)
    new_data_processed = preprocess_data(new_data)
    new_data_scaled, _, exog_features = prepare_features(new_data_processed)
    
    # Make predictions
    predictions = model.predict(
        start=0,  # Adjust start as necessary for your data
        end=len(new_data_scaled)-1,
        exog=new_data_scaled[exog_features]
    )
    
    # Rescale predictions if necessary
    predictions_rescaled = scaler.inverse_transform(
        np.hstack([new_data_scaled[exog_features], predictions.values.reshape(-1, 1)])
    )[:, -1]
    
    return predictions_rescaled


In [ ]:
model_results, metrics, fig, scaler = run_sarimax_analysis('/kaggle/input/beijing/Beijing.csv')

# Print metrics
print("\nModel Performance Metrics:")
print("=" * 50)
for metric, value in metrics.items():
    print(f"{metric:<15}: {value:.4f}")

# Display model summary
# print("\nModel Summary:")
# print("=" * 50)
# print(model_results.summary())

# Show interactive plots
fig.show()

Training SARIMAX model...


In [9]:
%time
model_results, metrics, fig, scaler = run_sarimax_analysis('/kaggle/input/beijing/Beijing.csv')

# Print metrics
print("\nModel Performance Metrics:")
print("=" * 50)
for metric, value in metrics.items():
    print(f"{metric:<15}: {value:.4f}")

# Display model summary
# print("\nModel Summary:")
# print("=" * 50)
# print(model_results.summary())

# Show interactive plots
fig.show()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 7.15 µs
Training SARIMAX model...
Model saved to sarimax_model.pkl
Making predictions...

Model Performance Metrics:
MSE            : 0.0825
RMSE           : 0.2873
MAE            : 0.2121
MAPE (%)       : 67.7428
R² Score       : 0.6849
